### Load backend project and export stereo rectification

In [ ]:
from pathlib import Path

from mynd.backend import metashape as backend

from mynd.api import GroupID, StereoCameraGroup
from mynd.camera import CameraCalibration
from mynd.geometry import (
    PixelMap,
    StereoRectificationResult,
    compute_stereo_rectification,
    rectify_image_pair,
)

from mynd.io import H5Database, create_file_database, load_file_database
from mynd.io import write_stereo_rectification_results

from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


def main():
    """Main entrypoint"""

    DOCUMENT_PATH: Path = Path(
        "/data/kingston_snv_01/acfr_revisits_metashape_dev/r23685bc_lite_version.psz"
    )
    DATABASE_PATH: Path = Path(
        f"/data/kingston_snv_01/acfr_frame_databases/r23685bc_20100605_021022_calibrations.h5"
    )

    # Load backend project
    load_project_result: Result[str, str] = backend.load_project(DOCUMENT_PATH)
    match load_project_result:
        case Ok(message):
            logger.info(message)
        case Err(message):
            logger.error(message)

    # Get the desired group from the backend project
    group_ids: list[GroupID] = backend.get_group_identifiers().unwrap()
    target: GroupID = group_ids[0]

    logger.info(f"Target: {target.key}, {target.label}")

    # Get stereo camera group
    stereo_group: StereoCameraGroup = (
        backend.get_stereo_cameras().unwrap().get(target)[0]
    )

    # Create or load database if path exists
    if DATABASE_PATH.exists():
        load_database_result: Result[H5Database, str] = load_file_database(
            DATABASE_PATH
        )
    else:
        load_database_result: Result[H5Database, str] = create_file_database(
            DATABASE_PATH
        )

    database = load_database_result.ok()

    # raise NotImplementedError

    # Calculate stereo rectification (intrinsics, pixel maps, homographies)
    rectification: StereoRectificationResult = compute_stereo_rectification(
        stereo_group.calibrations
    )

    # Write rectification to database
    write_result: Result[None, str] = write_stereo_rectification_results(
        database,
        group_name=f"/{target.label}/rectification",
        rectification=rectification,
    )

    if write_result.is_err():
        logger.error(write_result.err())

    # Visit all groups in the database and log them
    database.get("/").visit(lambda item: logger.info(item))


# ---------- Invoke main function ----------
main()